# Scene Video and Scanpath Mapping

In this tutorial, we will map gaze data from an eye-tracking recording to video frames, estimate a scanpath, and overlay the gaze fixations on the video. We will use the `pyneon` library to work with Neon eye-tracking recordings, which contain video and event data, including gaze information.

---

## Setup: Loading a Neon Recording

First, we load the Neon recording, which contains video and gaze data. Ensure that you have installed the required libraries such as `pyneon` and have the recording dataset available.

In [1]:
# Import necessary libraries
from pyneon import Dataset, get_sample_data

# Download sample data (if not existing) and return the path
sample_dir = get_sample_data("Artworks")
print(sample_dir)

dataset_dir = sample_dir / "Timeseries Data + Scene Video"
dataset = Dataset(dataset_dir)
print(dataset)

recording = dataset[0]
print(recording)

C:\Users\jan-gabriel.hartel\Documents\GitHub\PyNeon\data\Artworks
Dataset | 1 recordings

Recording ID: 9a141750-95ca-48ee-9693-53bbb896b87e
Wearer ID: c4f68887-e96c-467f-a901-0fc9fce09c0a
Wearer name: JGH
Recording start time: 2025-06-16 12:49:27.817000
Recording duration: 357.538s
                 exist                  filename                                                                                                                                        path
3d_eye_states     True         3d_eye_states.csv         C:\Users\jan-gabriel.hartel\Documents\GitHub\PyNeon\data\Artworks\Timeseries Data + Scene Video\artworks-9a141750\3d_eye_states.csv
blinks            True                blinks.csv                C:\Users\jan-gabriel.hartel\Documents\GitHub\PyNeon\data\Artworks\Timeseries Data + Scene Video\artworks-9a141750\blinks.csv
events            True                events.csv                C:\Users\jan-gabriel.hartel\Documents\GitHub\PyNeon\data\Artworks\Timeseries Data + S

In [2]:
import av

container = av.open(recording.video.video_file)
audio_stream = container.streams.audio[0]
print(audio_stream.rate)  # ← sampling rate

44100


---

## Mapping Gaze Data to Video Frames

In Neon recordings, gaze events are not naturally synchronized with the video. To map gaze data to specific video frames, we can use the `map_gaze_to_video` method. This method requires the `pyneon.video` object for determination of video timestamps, the `pyneon.fixations` object to make use of PupilLabs fixation detection pipeline and the `pyneon.gaze` object for improved time resolution of gaze estimation.

By default, Neon reports fixations with a single coordinate. This is computed as average between all gaze coordinates over the interval denoted as a fixation. However, this clashes with the functional definition of a fixation as _tracking a fixed point in space_, used by Neon.

Imagine looking at a fixed point, for example a street sign, while you are walking past it. Despite the movement of your body and the relative movement of the sign, the fixation will be stabilized. As such, taking an average gaze coordinate over the entire duration will not correspond to the location of the sign, or the fixation, at any given point in time. Feeding this point into an optical flow algorithm would, with high likelihood, lead to tracking anything but the sign.

Therefore, we use partial averages of gaze locations around the respective frame's timestamp. As the video is sampled at 30Hz while the gaze output nominally reaches 200Hz, we expect to take the average over 6 subsequent gaze points. This achieves a trade-off between recency of the reported gaze position at the given frame and error minimization, by averaging over microsaccades around the actual fixation target as well as random errors.

In [2]:
# Map gaze data to the video timestamps
synced_gaze = recording.sync_gaze_to_video(overwrite=True)

# Inspect the mapped gaze data
print(synced_gaze.data.tail())

                     gaze x [px]  gaze y [px]  worn  fixation id  blink id  \
timestamp [ns]                                                               
1750071325151422222          NaN          NaN  <NA>         <NA>      <NA>   
1750071325201422222          NaN          NaN  <NA>         <NA>      <NA>   
1750071325251422222          NaN          NaN  <NA>         <NA>      <NA>   
1750071325301422222          NaN          NaN  <NA>         <NA>      <NA>   
1750071325351422222          NaN          NaN  <NA>         <NA>      <NA>   

                     azimuth [deg]  elevation [deg]  frame_idx  
timestamp [ns]                                                  
1750071325151422222            NaN              NaN      10690  
1750071325201422222            NaN              NaN      10691  
1750071325251422222            NaN              NaN      10692  
1750071325301422222            NaN              NaN      10693  
1750071325351422222            NaN              NaN      10694 

Above, we can see that each frame gets a current gaze position as well as a fixation status. Currently, three types of fixation status are used:

1. `start` denoting the first frame corresponding to a fixation
2. `during` corresponding to intermediate frames of the same fixation
3. `end` denoting the last frame of the fixation

This determination will become relevant for tracking the scanpath with optical flow. After all, while a fixation is still active, we get up-to-date gaze information. Only after its end, tracking becomes necessary.

---

## 3. Estimating the Scanpath

Having matched every frame with a gaze coordinate, we can now get into the meat of the scanpath estimation. In dynamic scenes, the same object will not occupy the same scene-camera location over time. Therefore, we need to continuously map past fixation points as long as they are still visible in the frame.

The `estimate_scanpath` method achieves this by feeding fixation point denoted as `end` into a Lucas-Kanade sparse optical flow algorithm. This algorithm compares the video in vicinity of the point with the subsequent frame, updating the location in dependence of its movement. While a point is tracked, its status is flagged as `tracked`. In practice, many scene frames will have multiple simultaneously present past fixations. Our implementation carries them and repeately performs an optical flow estimation for each point. Only when they can no longer be tracked, will they be flagged as `lost` and subsequently dropped for the next frame.

It should be noted that this algorithm is not optimised for performance and that it will take a considerable amount of time to run on limited hardware. For our computers, the algorithm takes roughly half the time of the video, though this benchmark heavily depends on the density of past fixation points and computational ressources


In [ ]:
# Estimate the scanpath based on the mapped gaze data
from pyneon.video import estimate_scanpath

scanpath_df = estimate_scanpath(recording.scene_video, synced_gaze)
scanpath_df.index.name = "timestamp [ns]"
scanpath = Stream(scanpath_df)

# Inspect the estimated scanpath
print(scanpath.data.head())

Loading saved scanpath from C:\Users\jan-gabriel.hartel\Documents\GitHub\PyNeon\data\Artworks\Timeseries Data + Scene Video\artworks-9a141750\derivatives\scanpath.pkl
                                                             fixations  \
timestamp [ns]                                                           
1750070967817000000    fixation id gaze x [px] gaze y [px] fixation...   
1750070967867000000    fixation id gaze x [px] gaze y [px] fixation...   
1750070967917000000    fixation id gaze x [px] gaze y [px] fixation...   
1750070967967000000    fixation id gaze x [px] gaze y [px] fixation...   
1750070968017000000    fixation id gaze x [px] gaze y [px] fixation...   

                     frame_idx  
timestamp [ns]                  
1750070967817000000          0  
1750070967867000000          1  
1750070967917000000          2  
1750070967967000000          3  
1750070968017000000          4  


We should take a moment to understand the format of the `scanpath.data`. As we care about getting a scanpath mapped on every single video-frame, we create it as a dataframe of dataframes. As such, every row carries both the timestamp as well as the frame index of the underlying video and saves a dataframe in the `fixations` cell. In this dataframe, every present fixation is provided with an id, coordinates and a fixation status, as seen below. The benefit of treating is a dataframe is the possibility to use intuitive pandas indexing, allowing us, for example, to get a list of fixations at frame 2000.

As a quirk of Neon taking some time to start up, the first frames will usually not yield any usable results. Still, we carry them for consistency. 

In [4]:
# print fixations when column frame_idx is 1334. Frame_idx is not the idx of the dataframe, but the index of the video frame.
print(scanpath.data.loc[scanpath.data["frame_idx"] == 2000, "fixations"].values[0])

   fixation id gaze x [px] gaze y [px] fixation status
0          174  903.107571  428.153429          during
1          173  924.962036  437.257629         tracked
2          172  922.353638  493.392975         tracked
3          171  960.446289  465.010101         tracked
4          170  835.920654  440.765076         tracked
5          169  833.580444  467.248718         tracked
6          168   698.10437  662.543335         tracked
7          167  693.992065  630.123047         tracked
8          166  713.760315  561.713867         tracked
9          165  543.353333  473.653381         tracked
10         164  698.986816   486.05957         tracked
11         163  704.265442  439.653992         tracked
12         162  750.842773  539.532349         tracked


---

## 4. Understanding Fixation Status

Each fixation is assigned a status that indicates its lifecycle:

- **start**: first frame of fixation
- **during**: intermediate frames of fixation
- **end**: last frame of fixation
- **tracked**: Optical flow algorithm tracks fixation
- **lost**: Tracking is lost, fixation is no longer tracked and gets dropped

---

## 5. Overlaying Fixations on the Video

Now that we have the scanpath, we can overlay the gaze fixations on the video. This creates a video output with overlaid fixations, where:

- A **blue dot** represents the current gaze location.
- **Green dots** represent tracked fixations.
- A **red dot** indicates no fixation (saccades or blinks).

Further, we draw connecting lines between past fixations to show the scanpath for the current video. The show_video option creates a live-output of the video rendering, but also increases the runtime.

In [ ]:
# Overlay the scanpath on the video and show the output
from pyneon.vis import overlay_scanpath

overlay_scanpath(
    recording.scene_video,
    scanpath.data,
    circle_radius=10,
    line_thickness=2,
    text_size=1,
    max_fixations=10,
    show_video=True,
    output_path=None,
)

Loading saved scanpath from C:\Users\jan-gabriel.hartel\Documents\GitHub\PyNeon\data\Artworks\Timeseries Data + Scene Video\artworks-9a141750\derivatives\scanpath.pkl
Overlay video already exists at C:\Users\jan-gabriel.hartel\Documents\GitHub\PyNeon\data\Artworks\Timeseries Data + Scene Video\artworks-9a141750\derivatives\scanpath.mp4; skipping render.
`show_video=True` has no effect because rendering was skipped.


---

## Summary

- **Mapping Gaze to Video**: We used the `map_gaze_to_video` method to match gaze data with video frames based on timestamps.
- **Estimating Scanpath**: The scanpath was estimated using `pyneon.video.estimate_scanpath`, which tracks fixations and uses optical flow to follow past fixations across scene changes.
- **Overlaying Fixations**: The fixations were visualized on the video by calling `pyneon.vis.overlay_scanpath`.

This workflow can be used to process eye-tracking data, align it with video frames, and visualize gaze movements within video recordings.
